# Merged results filtering - part 2.

We decided to try a better filtering, that is also very restrictive, based on removing a full CRISPR cluster if it contains spacers flanked by common K-mers.

Problems:
- 1) CRASS mistakenly classifies direct, but degraded, repeats as CRISPRs.

See JDotter plot ran with window size 20 bases on a read bellonging to group 1907. A dark dot means the window centered at that position is perfectly repeated. The mid-diagonal is just the read plotted against itself without any offset, but their is also a parallel line that is offset by ca 30 bp, i.e. a repeat. If you blast the first 30 bases of the read against the full read it looks like this (query is the first 30 bp, sbjct is the read):

```
Query  1   CGGCCCCCTGGCCTGCCTAGCCGCAGCGCTCG  32
           ||||||||||||||||||||||||||||||||
Sbjct  1   CGGCCCCCTGGCCTGCCTAGCCGCAGCGCTCG  32
```
>First it seems to sometimes misidentify imperfect direct repeats as CRISPRs (but in fact they are something else, there are many types of repeats in genomes). This can happen if the repeat is degraded so that some parts of the repeat differ by some bases between repeated units. Other parts may be perfectly repeated. CRASS will think that the perfectly repeated regions are CRISPR repats and the intermediate pieces are spacers.  

> Hence, an imperfect repeat. In the attached rtf file I've aligned the CRISPR repeat (defined by crass), one spacer and the same read as before. The green and yellow are the imperfect direct repeats. As you see both the CRISPR repeat and the spacer includes the repeat. In this case there only seems to be two repetitions of the repeated sequence in the read. Maybe this specific repeat only exist in this configuration (but it can still be copied in many places in the genome as di-repeats), or we can find it more copies on other reads, haven't checked in detail. The main problem with making a problem for finding CRISPRs is exactly this, to avoid wrongly assigning non-CRISPR repeats as CRISPRs. I didn't have good solution for that when I made my own program for CRISPR finding before (in https://www.ncbi.nlm.nih.gov/pubmed/18497291), that's why I didn't publish that program independently. Back then we were using Sanger data and didn't get that many CRISPR clusters so I manually looked at the ones we found to remove false assignments.

- 2) CRASS only uses the first or last n bases of the spacers to decide on which reads they are found.

> When CRASS initially identifies a spacer it requires that it is surrounded by repeats on both sides. But after that when the p-graph is built it just uses the n first or last bases of the spacers to decide on which reads they are located. This generally works well, but when multiple spacers share n bases in the beginning or end it of course screws up. That's what we are seeing - all problematic regions (also the one you talked about today) have spacers that share the beginning or the end and therefor CRASS maps them to the same sequence reads. That's why we had 18 spacers mapping (according to CRASS) to the same sequence read in the example I gave before. When doing manual alignments the whole spacers don't usually align, but the first or last n bases do.

One thing that we perhaps could try is to remove all CRISPR clusters where any pair of spacers share n bases in the beginning or the end. n should be the same length as CRASS uses when building the p-graph. I also noticed that a region of the repeat sometimes are shared with a region of spacers of the same CRISPR cluster for these problematic clusters. We could also remove those cases, i.e. ***remove clusters where a spacer shares the first or last n-bases with an n-mer in the repeat (anywhere in the repeat)***.

In [3]:
%matplotlib inline
import sys
import pandas as pd
import matplotlib
matplotlib.style.use('ggplot')
import seaborn as sns
sns.set(color_codes=True)

# my laptop config:
# source code location
src = "/home/sergiu/data/work/andersson/src/andersson/src"
# crispr file generated by crass (xml format)
crispr_location = "/home/sergiu/data/work/andersson/data/asko/crass/crass.crispr"
# tab separated output
crispr_tab_location = "/home/sergiu/data/work/andersson/data/asko/spacer_table_trimmed.txt"

if not src in sys.path:
    sys.path.append(src)
else:
    print(sys.path)

import crass_crispr_parser as crispr

['', '/home/sergiu/programs/miniconda3/envs/andersson/lib/python36.zip', '/home/sergiu/programs/miniconda3/envs/andersson/lib/python3.6', '/home/sergiu/programs/miniconda3/envs/andersson/lib/python3.6/lib-dynload', '/home/sergiu/programs/miniconda3/envs/andersson/lib/python3.6/site-packages', '/home/sergiu/programs/miniconda3/envs/andersson/lib/python3.6/site-packages/IPython/extensions', '/home/sergiu/.ipython', '/home/sergiu/data/work/andersson/src/andersson/src']


In [4]:
import crass_crispr_parser as crispr

doc = crispr.import_crass_crispr(crispr_location)
groups, spacers, sources = crispr.extract_info_all(doc)

Done!


881

In [12]:
print(sum([len(spacers[i]) for i in spacers]))
for gid in spacers:
    group = spacers[gid]
    torem = set()  # spacer ids to be removed
    for spid1 in group:
        # record all spacers similar to this one in the first nine characters
        similar = set()
        for spid2 in group:
            if group[spid1]['seq'][:9] == group[spid2]['seq'][:9] or \
            group[spid1]['seq'][9:] == group[spid2]['seq'][9:]:
                similar.add(spid2)
        if len(similar)>1:
            torem |= similar
    # remove this spacer if there are other similar spacers
    for spid in torem:
        del group[spid]
print(sum([len(spacers[i]) for i in spacers]))

5245
4255


In [14]:
# remove clusters where a spacer shares the first or last n-bases 
# with an n-mer in the repeat (anywhere in the repeat).
print(sum([len(spacers[i]) for i in spacers]))
for gid in spacers:
    group = spacers[gid]
    torem = set()
    drseq = groups[gid]
    for spid in group:
        spseq = group[spid]['seq']
        if spseq[:9] in drseq or spseq[:9] in drseq:
            torem.add(spid)
    for spid in torem:
        del group[spid]
print(sum([len(spacers[i]) for i in spacers]))

4255
3942


In [16]:
# just to figure out what's what:
print(spacers[list(spacers.keys())[0]])
print(sources[list(spacers.keys())[0]])
print(groups[list(spacers.keys())[0]])

{'SP54': {'seq': 'ATGTAGGCTCACCATTGGCATTGAAGGTC', 'soids': {'SO25', 'SO48', 'SO8', 'SO40', 'SO33', 'SO17', 'SO42', 'SO49', 'SO51', 'SO43', 'SO44', 'SO21', 'SO16', 'SO50', 'SO20', 'SO2', 'SO24', 'SO45', 'SO38', 'SO23', 'SO27', 'SO13', 'SO14', 'SO15', 'SO41', 'SO53', 'SO37', 'SO7', 'SO52', 'SO47', 'SO12', 'SO39', 'SO26', 'SO19', 'SO18', 'SO9', 'SO46', 'SO22', 'SO36'}}, 'SP6': {'seq': 'TCGCGGATTGCCGCCGTGCTGGCGTTGAT', 'soids': {'SO25', 'SO16', 'SO27', 'SO8', 'SO12', 'SO13', 'SO14', 'SO17', 'SO26', 'SO15', 'SO19', 'SO20', 'SO2', 'SO18', 'SO24', 'SO9', 'SO22', 'SO23', 'SO7', 'SO21'}}, 'SP32': {'seq': 'GTGCTCATCTTTGTTGAAGTGAAGCCCGA', 'soids': {'SO28'}}}
{'SO2': 'SRR3727523_R1:SRR3727523.56265', 'SO7': 'SRR3727523_R1:SRR3727523.1643880', 'SO8': 'SRR3727523_R1:SRR3727523.3135102', 'SO9': 'SRR3727523_R1:SRR3727523.10110063', 'SO12': 'SRR3727523_R1:SRR3727523.11078039', 'SO13': 'SRR3727523_R1:SRR3727523.16547319', 'SO14': 'SRR3727523_R1:SRR3727523.16863415', 'SO15': 'SRR3727523_R1:SRR3727523.2196